# Creating a task in Dime.Scheduler with the SDK

Check out the [📚 official docs](https://docs.dimescheduler.com/) for more information.

## What we will do

This notebook is an interactive guide that will you show you to create a task in Dime.Scheduler. 

## What is a task

A **task** is a unit of work that needs to be scheduled, and as such it could be anything:

- Send a technician to the customer to fix the broken A/C
- Assign a consultant to a project that you just sold
- Manufacture a batch of orange juice
- Assemble a bicycle
- Rent out a piece of equipment

**Tasks** are the low-level units of work, and they always belong to a **job**, which is the Dime.Scheduler equivalent of a service order, project, assembly order, sales order, etc. Jobs container master data such as customer information, contact information, addresses, etc.

In what follows is a step-by-step guide that shows you how to use the SDK to create jobs and tasks. 

## Step 1: Retrieving the bits and bytes

All logic to interact with Dime.Scheduler is contained in the `Dime.Scheduler.Sdk` NuGet package, which is available [here](https://www.nuget.org/packages/Dime.Scheduler.Sdk/). The instruction below will fetch this package, which we will invoke once it's been downloaded.

In [ ]:
#r "nuget:Dime.Scheduler.Sdk, 2.0.0-alpha.18"

Installed Packages Dime.Scheduler.Sdk, 1.0.0-beta.12

## Step 2: Adding references to the SDK

Now that the package is loaded, we can start having some fun.  We'll start by adding a reference to the SDK in the code, so the compiler knows what we're talking about:

In [ ]:
using Dime.Scheduler.Sdk;

## Step 3: Instantiating the DimeSchedulerClient class

The only requirement is to have a Dime.Scheduler API, which you can generate in the app. 
An optional override in `DimeSchedulerClient` makes it possible to change environment.

In [1]:
string key = "DS-49WHSIU3W-6QQ6C3Z8-QVSJRG2P-PVI6HW";
DimeSchedulerClient client = new DimeSchedulerClient("https://test.api.dimescheduler.com", key);

## Step 4: Constructing the request

The SDK contains all the models you can interact with as C# classes. Just pick the model of interest and instantiate it with the right set of properties.

To create a task, we actually need to provide two entities:

- The **job**, which holds a set of tasks. Think of a job as a project and a task of one of the many activities that need to be done to finish the project
- The *task* itself, which must refer to a job that is already available in Dime.Scheduler

A job, therefore, is not always necessary. If the job is already available, you may skip step 5A, but make sure to use the correct `JobNo` value in step 5B.

### Step 4A: Creating a job

The job model contains a lot of information. So much that we've created a `JobBuilder` class to facilitate the creation of these objects. 
Most of the fields speak for themselves, although we should highlight the following three fields:

- **Source App**: this is a unique identifier that tells to which back-office system this job (and the subsequent tasks) belongs to.
- **Source Type**: this is a unique identifier that tells the underlying back-office system what module this job (and the subsequent tasks) belongs to
- **Job No**: this is a unique identifier which must exist in Dime.Scheduler before a task can be created.

In [1]:
using Dime.Scheduler.Sdk.Import;
using Dime.Scheduler.Sdk.Import.Builder;

string sourceApp = "NOTEBOOK";
string sourceType = "SERVICE";

Job newJobToCreate = new JobBuilder()
    .WithJob(sourceApp, sourceType, "NB0001", "Yearly maintenance & repair job", "Yearly service", "Simon Pecker")
    .WithCustomer(new Customer
    {
        Email = "hq@customer.com",
        Name = "HQ London",
        No = "HQ_LONDON",
        Phone = "123 423 091",
        Reference = "REF"
    })
    .Create();

The last step is to invoke the request and let the API do its job.

In [1]:
await client.Jobs.CreateAsync(newJobToCreate);

### Step 4B: Creating a task

Once the job is created, we can go ahead and link a task to that job. It is important that the source app, type, and job no link to the right job. And lastly, the **Task No** field is an unique identifier that can be used by both Dime.Scheduler and the back-office system so they understand this is the same task.

In [1]:
Task newTaskToCreate = new Task() 
{
	SourceApp = sourceApp,
	SourceType = sourceType,
	JobNo = "NB0001", // There must be a job in the database that holds this value
	TaskNo = "NO0001_2022_ROOM6",
	Description= "Service air conditioning in room #6",
	ShortDescription= "Service air conditioning in room #6"
};

And finally, invoke the request:

In [1]:
await client.Tasks.CreateAsync(newJobToCreate);

So there you go, you've just created a task in Dime.Scheduler and should be now available in the open task list. So now it's time to stop puzzling and start planning!